In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances

### Idea is to model 990s and 990EZs separately and then use a similarity metric to combine like clusters.

In [2]:
df990 = pd.read_csv('./sample_data_2017_M4_990_labels.csv')
dfEZ = pd.read_csv('./sample_data_2017_M4_990EZ_labels.csv')

df_all = pd.read_csv('../../sample_data_2017_M4_cleaned.csv')

FileNotFoundError: File b'./sample_data_2017_M4_990_labels.csv' does not exist

In [161]:
def standardize(df):
    eins = list(df['EIN'])
    df = df.fillna(0)
    cols = df.loc[:, df.columns != 'EIN'].columns
    df = pd.DataFrame(preprocessing.scale(df.loc[:, df.columns != 'EIN']))
    df.columns = cols
    df['EIN'] = eins
    return df

In [162]:
## need to standardize across all documents, not within groups so that distance is accurate for entire space
df_all = df_all[['GrossReceiptsAmt', 'RevenueAmt', 'TotalAssetsGrp/EOYAmt',
       'TotalNetAssetsFundBalanceGrp/EOYAmt', 'EIN']]
df_all = standardize(df_all)
df_all.head()

,GrossReceiptsAmt,RevenueAmt,TotalAssetsGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt,EIN
0,-0.020728,-0.024919,-0.036006,-0.033420,10021545
1,-0.022250,-0.023062,-0.032762,-0.024127,10024645
2,-0.024067,-0.024919,-0.037833,-0.033388,10029480
3,-0.023345,-0.024919,-0.036625,-0.031158,10039552
4,-0.021403,-0.024919,-0.035809,-0.031670,10043280


In [163]:
df990 = df990[['EIN', 'model_group', 'labels']].merge(df_all, on = 'EIN', how = 'left')
dfEZ = dfEZ[['EIN', 'labels']].merge(df_all, on = 'EIN', how = 'left')

In [120]:
means990 = pd.DataFrame(df990.groupby(['labels', 'model_group']).mean())
means990.reset_index(inplace = True)
means990 = means990.drop(['EIN'], axis = 1)
means990.head()

,labels,model_group,GrossReceiptsAmt,RevenueAmt,TotalAssetsGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt
0,1,0,-0.023071,-0.024547,-0.031946,-0.023167
1,1,1,-0.011659,-0.023892,-0.010209,0.016163
2,1,2,-0.004692,-0.003269,-0.000762,0.021850
3,1,3,0.071198,-0.002608,0.049625,0.064951
4,2,0,-0.022792,-0.024918,-0.035446,-0.029047


In [121]:
meansEZ = pd.DataFrame(dfEZ.groupby(['labels']).mean())
meansEZ.reset_index(inplace = True)
meansEZ = meansEZ.drop(['EIN'], axis = 1)
meansEZ.head()

,labels,GrossReceiptsAmt,RevenueAmt,TotalAssetsGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt
0,0,-0.024173,-0.024130,-0.037888,-0.033532
1,1,-0.024432,-0.024781,-0.037686,-0.033146
2,2,-0.024358,-0.024619,-0.036869,-0.031643
3,3,-0.024240,-0.024358,-0.037281,-0.032442
4,4,-0.024191,-0.024634,-0.037851,-0.033438


In [122]:
x990 = means990.values
xEZ = meansEZ.values

In [123]:
# for clusterEZ in meansEZ:
#     most_similar = []
#     similarity_scores = []
# for cluster990 in means990:
    
similarity_scores = euclidean_distances(xEZ[:,1:], x990[:,2:])

In [125]:
min_distance = []
position = []
for row in similarity_scores:
    min_distance.append(min(row))
    position.append(min(enumerate(row),key=lambda x: x[1])[0])

In [127]:
meansEZ['min_distance'] = min_distance
meansEZ['position'] = position

In [138]:
combined_clusters = meansEZ.merge(means990, how = 'left', left_on = 'position', right_index = True)
combined_clusters

,labels_x,GrossReceiptsAmt_x,RevenueAmt_x,TotalAssetsGrp/EOYAmt_x,TotalNetAssetsFundBalanceGrp/EOYAmt_x,min_distance,position,labels_y,model_group,GrossReceiptsAmt_y,RevenueAmt_y,TotalAssetsGrp/EOYAmt_y,TotalNetAssetsFundBalanceGrp/EOYAmt_y,final_labels
0,0,-0.024173,-0.024130,-0.037888,-0.033532,0.000483,156,40,0,-0.023974,-0.024188,-0.037626,-0.033184,0.40
1,1,-0.024432,-0.024781,-0.037686,-0.033146,0.000344,52,14,0,-0.024137,-0.024919,-0.037577,-0.033130,0.14
2,2,-0.024358,-0.024619,-0.036869,-0.031643,0.000424,124,32,0,-0.024276,-0.024728,-0.037088,-0.031980,0.32
3,3,-0.024240,-0.024358,-0.037281,-0.032442,0.000282,678,201,0,-0.024154,-0.024189,-0.037412,-0.032604,0.201
4,4,-0.024191,-0.024634,-0.037851,-0.033438,0.000322,72,19,0,-0.024339,-0.024919,-0.037871,-0.033460,0.19
5,5,-0.024432,-0.024747,-0.037937,-0.057794,0.012355,153,39,1,-0.021214,-0.016432,-0.035344,-0.049645,1.39
6,6,-0.024079,-0.023867,-0.037833,-0.033455,0.000481,156,40,0,-0.023974,-0.024188,-0.037626,-0.033184,0.40
7,7,-0.024266,-0.024353,-0.037896,-0.033534,0.000556,156,40,0,-0.023974,-0.024188,-0.037626,-0.033184,0.40
8,8,-0.024399,-0.024706,-0.037320,-0.032489,0.000429,48,13,0,-0.024349,-0.024533,-0.037530,-0.032816,0.13
9,9,-0.024094,-0.023923,-0.037426,-0.032726,0.000165,476,130,0,-0.024079,-0.023835,-0.037398,-0.032590,0.130


### Most are closest to clusters in the 0 group, i.e. lowest GrossReceiptAmt

#### Need to find a more scientific method for determining threshold for when to combine and when to leave separate.

In [137]:
threshold = .1
final_labels = []
for i in combined_clusters.index:
    if combined_clusters.loc[i, 'min_distance'] < threshold:
        final_labels.append(str(combined_clusters.loc[i, 'model_group']) + '.' + str(combined_clusters.loc[i, 'labels_y']))
    else:
        final_labels.append('4.' + combined_clusters.loc[i, 'labels_x'])
combined_clusters['final_labels'] = final_labels

In [164]:
df990['final_labels'] = [str(df990.loc[i, 'model_group']) + '.' + str(df990.loc[i, 'labels']) for i in df990.index]
df990 = df990.drop(['model_group', 'labels'], axis = 1)

In [165]:
dfEZ = dfEZ.merge(combined_clusters[['labels_x', 'final_labels']], how = 'left', left_on = 'labels', right_on = 'labels_x')
dfEZ = dfEZ.drop(['labels', 'labels_x'], axis = 1)

In [166]:
dfEZ.head()

,EIN,GrossReceiptsAmt,RevenueAmt,TotalAssetsGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt,final_labels
0,10180687,-0.024228,-0.024220,-0.037803,-0.033332,0.40
1,10192722,-0.024246,-0.024536,-0.037869,-0.033697,0.19
2,10216941,-0.024196,-0.024138,-0.037351,-0.032492,0.201
3,10217620,-0.024402,-0.024671,-0.037399,-0.032568,0.13
4,10226245,-0.024131,-0.024096,-0.037754,-0.033320,0.40


In [167]:
df990.head()

,EIN,GrossReceiptsAmt,RevenueAmt,TotalAssetsGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt,final_labels
0,206207435,-0.023890,-0.024919,-0.037245,-0.032308,0.77
1,61424632,-0.023407,-0.023625,-0.031268,-0.021159,0.173
2,10618114,-0.023891,-0.024919,-0.035892,-0.029718,0.50
3,411758837,-0.024071,-0.024919,-0.036455,-0.031020,0.86
4,260289115,-0.023554,-0.024919,-0.037050,-0.032581,0.50


In [168]:
df_final = df990.append(dfEZ)
df_final.head()

,EIN,GrossReceiptsAmt,RevenueAmt,TotalAssetsGrp/EOYAmt,TotalNetAssetsFundBalanceGrp/EOYAmt,final_labels
0,206207435,-0.023890,-0.024919,-0.037245,-0.032308,0.77
1,61424632,-0.023407,-0.023625,-0.031268,-0.021159,0.173
2,10618114,-0.023891,-0.024919,-0.035892,-0.029718,0.50
3,411758837,-0.024071,-0.024919,-0.036455,-0.031020,0.86
4,260289115,-0.023554,-0.024919,-0.037050,-0.032581,0.50


In [171]:
df_final.to_csv('./sample_data_2017_M4_ALL_labels.csv', index = False)